In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)
from sklearn.impute import SimpleImputer

In [2]:
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity, cosine_distances
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [3]:
#reading in my data frame and replacing spaces and unwanted marks in the columns
recommend = pd.read_csv('./data/recommended_tableau.csv')
recommend.drop(columns=['Unnamed: 0'], inplace=True)
reco_names = recommend[['fips','state', 'name']]

ss = StandardScaler()
reco_scaled = ss.fit_transform(recommend.drop(columns=['fips','name','state']))

cs = cosine_similarity(reco_scaled)
recommended = pd.DataFrame(reco_scaled)
recommended = pd.concat([reco_names, recommended], axis=1, ignore_index=True)
recommended = recommended.rename(columns={0: 'fips', 1: 'name', 2: 'state'})

In [4]:
recommend.head(2)

,fips,state,name,pop2010,age_under_5_2010,age_under_18_2010,age_over_65_2010,female_2010,white_2010,two_plus_races_2010,hispanic_2010,white_not_hispanic_2010,no_move_in_one_plus_year_2010,foreign_born_2010,foreign_spoken_at_home_2010,hs_grad_2010,bachelors_2010,veterans_2010,mean_work_travel_2010,housing_units_2010,homeownership_2010,housing_multi_unit_2010,median_val_owner_occupied_2010,households_2010,persons_per_household_2010,per_capita_income_2010,median_household_income_2010,private_nonfarm_establishments_2009,private_nonfarm_employment_2009,building_permits_2010,area_2010,density_2010,poverty_2010,age_over_18_2019,age_over_65_2019,age_over_85_2019,age_under_5_2019,asian_2019,avg_family_size_2019,bachelors_2019,black_2019,hispanic_2019,household_has_broadband_2019,household_has_computer_2019,household_has_smartphone_2019,households_2019,households_speak_asian_or_pac_isl_2019,households_speak_limited_english_2019,households_speak_other_2019,households_speak_other_indo_euro_lang_2019,households_speak_spanish_2019,housing_mobile_homes_2019,housing_one_unit_structures_2019,housing_two_unit_structures_2019,hs_grad_2019,mean_household_income_2019,median_age_2019,median_household_income_2019,median_individual_income_2019,median_individual_income_age_25plus_2019,native_2019,other_single_race_2019,pac_isl_2019,per_capita_income_2019,persons_per_household_2019,pop_2019,two_plus_races_2019,unemployment_rate_2019,uninsured_2019,uninsured_65_and_older_2019,uninsured_under_19_2019,uninsured_under_6_2019,veterans_2019,white_2019,white_not_hispanic_2019
0,1001,Alabama,Autauga County,54571,6.6,26.8,12.0,51.3,78.5,1.6,2.4,77.2,86.3,2.0,3.7,85.3,21.7,5817,25.1,22135,77.5,7.2,133900,19718,2.7,24568,53255,877,10628,191,594.44,91.8,10.6,76.2,15.0,1.6,5.8,1.0,3.09,26.6,19.0,2.8,80.6,73.0,78.4,21397,1.8,0.7,0.2,0.3,2.9,26.7,17.3,73.3,88.5,75326,38.2,58731,29725,40778,0.3,0.7,0.0,29819,2.56,55380,2.2,3.5,7.1,0.0,1.7,1.7,12.6,76.8,74.6
1,1003,Alabama,Baldwin County,182265,6.1,23.0,16.8,51.1,85.7,1.5,4.4,83.5,83.0,3.6,5.5,87.6,26.8,20396,25.8,104061,76.7,22.6,177200,69476,2.5,26469,50147,4812,52233,696,1589.78,114.6,12.2,78.3,20.0,1.9,5.5,0.9,3.24,31.9,9.3,4.6,81.8,76.3,81.7,80930,0.6,1.2,0.0,1.8,4.6,24.8,11.5,75.2,90.8,80986,43.0,58320,29802,37897,0.8,1.1,0.0,32626,2.59,212830,1.7,4.0,8.9,0.3,3.8,2.2,11.8,86.2,83.1


In [5]:
countyInput = 'Montgomery County'
stateInput =  'Maryland'
state = recommend[recommend['state'] == stateInput]
countyIndex = state[recommend.name == countyInput].index.values[0]
countyIndex

/var/folders/w3/8j3br3pn37q4dc1yqw7571580000gn/T/ipykernel_9946/1569479478.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  countyIndex = state[recommend.name == countyInput].index.values[0]


1208

In [8]:

def similar_county(countyInput, stateInput):
    state = recommend[recommend['state'] == stateInput]
    countyIndex = state[recommend.name == countyInput].index.values[0]
    scores = list(enumerate(cs[int(countyIndex)]))
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    sorted_scores = sorted_scores[1:]
    
    j = 0
    print("Based on Cosine Similarities, the 10 most recommended products to product",
          countyInput,',', stateInput, 'are:\n')
    
    for item in sorted_scores:
        similar_d = item[0]
        distance = item[1]
        county_ = recommended.iloc[similar_d][['name', 'state']].values[0]
        state_ = recommended.iloc[similar_d][['name', 'state']].values[1]

        print(j+1, ')', state_, ',', county_,
              'with a similarity score of', round(distance, 3), 'out of 1')
        j = j+1
        if j > 4:
            break
    
    


In [9]:
similar_county(countyInput, stateInput)


Based on Cosine Similarities, the 10 most recommended products to product Montgomery County , Maryland are:

1 ) Fairfax County , Virginia with a similarity score of 0.977 out of 1
2 ) Bergen County , New Jersey with a similarity score of 0.965 out of 1
3 ) Contra Costa County , California with a similarity score of 0.958 out of 1
4 ) Lake County , Illinois with a similarity score of 0.953 out of 1
5 ) DuPage County , Illinois with a similarity score of 0.94 out of 1


/var/folders/w3/8j3br3pn37q4dc1yqw7571580000gn/T/ipykernel_9946/2752810222.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  countyIndex = state[recommend.name == countyInput].index.values[0]
